This notebook may possible charge you lots of money.
Use with care.

In [33]:
import googlemaps
from datetime import datetime, timedelta
from pprint import pprint
import pytz
import os
import json
import numpy as np
import itertools

Load configuration

In [34]:
base_path = os.path.join('C:\\', 'Users', 'glenn', 'src', 'pycommute')
data_directory = os.path.join(base_path, 'data')
data_file = os.path.join(data_directory, 'test_data.json')
use_dummy = True
if use_dummy:
    config_file = os.path.join(base_path, 'edit_this.config.json')
else:
    config_file = os.path.join(base_path, 'config.json')
with open(config_file, 'r') as f:
    config = json.load(f)

Get API key from file

In [35]:
with open(os.path.join(base_path, 'api_key.txt'), 'r') as f:
    api_key = f.read().strip()

Calculate arrival time

In [36]:
local_tz = pytz.timezone(config['time_zone'])
arrival_time = datetime.now(local_tz).replace(hour=8, minute=0, second=0, microsecond=0) + timedelta(days=1) # Tomorrow at 8
while arrival_time.weekday() != 0: # Finding the next Monday.
    arrival_time += timedelta(days=1)
    
print(f'Calculating for arrival at {arrival_time}.')

Calculating for arrival at 2020-04-27 08:00:00+02:00.


Initialize API

In [37]:
gmd = googlemaps.Client(key=api_key) # Data API

Define function to calculate a point from a origin and distance and heading.

In [38]:
def point_from_distance(lat0d, lon0d, distance, heading):
    if heading == 'north':
        psi = 0
    elif heading == 'east':
        psi = -np.pi/2
    elif heading == 'south':
        psi = np.pi
    elif heading == 'west':
        psi = np.pi/2
    else:
        psi = np.radians(heading)
    
    G = 6371000 # great circle radius
    d = distance / G
    
    lat0 = np.radians(lat0d)
    lon0 = np.radians(lon0d)
    
    lat = np.arcsin(np.sin(lat0) * np.cos(d) + np.cos(lat0) * np.sin(d) * np.cos(psi))
    if abs(np.cos(lat)) <= 0.001:
        lon = lon0
    else:
        lon = ((lon0 - np.arcsin(np.sin(psi) * np.sin(d) / np.cos(lat)) + np.pi) % (2*np.pi)) - np.pi
    
    return np.degrees(lat), np.degrees(lon)

Create grid

In [39]:
lat_center = (config['northeast'][0]+config['southwest'][0])/2
lon_center = (config['northeast'][1]+config['southwest'][1])/2
dlat = point_from_distance(lat_center, lon_center, config['resolution'], 'north')[0] - lat_center
dlon = point_from_distance(lat_center, lon_center, config['resolution'], 'east')[1] - lon_center
latitudes = np.arange(config['southwest'][0], config['northeast'][0], dlat)
longitudes = np.arange(config['southwest'][1], config['northeast'][1], dlon)
origins = list(itertools.product(latitudes, longitudes))
destinations_geocodes = [gmd.geocode(destination)[0] for destination in config['destinations']]
destinations = [(gc['geometry']['location']['lat'], gc['geometry']['location']['lng']) for gc in destinations_geocodes]
print(f'Number of origins: {len(origins)}, number of destinations: {len(config["destinations"])}. {len(config["destinations"]) * len(origins) * 2} elements is approximately {len(config["destinations"]) * len(origins) * 2 * 0.005} USD.')

Number of origins: 1426, number of destinations: 2. 5704 elements is approximately 28.52 USD.


Since the distance_matrix api can only handle requests with maximum 25 origins and destinations, we split the origins into batches of 25.

In [40]:
max_distance_matrix_locations = 25
i_origins = iter(origins)
origins_batches = []
while True:
    next_batch = list(itertools.islice(i_origins, max_distance_matrix_locations))
    if not next_batch:
        break
    origins_batches.append(next_batch)

We also create a function that takes a list of batches, options, then calls the API and returns the results.

In [41]:
def call_distance_matrix_batched(origins_batches, **kwargs):
    distance_matrices = []
    for origins in origins_batches:
        distance_matrix = gmd.distance_matrix(origins=origins, **kwargs)
        distance_matrices.append(distance_matrix)
    return distance_matrices

Options that stay the same

In [42]:
static_args = {
    'language': config.get('language', 'english'),
    'units': config.get('units', 'metric'),
    'destinations': destinations,
    'arrival_time': arrival_time,
}

Calculate distance with transit

In [43]:
distance_matrices_transit = call_distance_matrix_batched(origins_batches, mode='transit', **static_args)

Calculate distances with driving

In [44]:
distance_matrices_driving = call_distance_matrix_batched(origins_batches, mode='driving', **static_args)

View data

Store data

In [46]:
data_object = {
    'distance_matrices_transit': distance_matrices_transit,
    'distance_matrices_driving': distance_matrices_driving,
    'origins': origins,
    'origins_batches': origins_batches,
    'destinations_geocodes': destinations_geocodes,
    'config': config,
}

with open(data_file, 'w') as f:
    json.dump(data_object, f, indent=4)
